### XGBoost PIPELINE

In [1]:
import joblib
from datetime import datetime
import numpy as np
import os
from sklearn.model_selection import train_test_split

import sys
sys.path.insert(0, '/Users/florianrunkel/Documents/02_Uni/04_Masterarbeit/masterthesis/')

from ml_pipe.data.database.mongodb import MongoDb
from ml_pipe.data.featureEngineering.featureEngineering import featureEngineering

from ml_pipe.models.xgboost.model import XGBoostModel

In [4]:
def run_xgboost_pipeline():
    print("Starte XGBoost-Pipeline...")

    # Daten aus MongoDB holen
    mongo = MongoDb()
    raw_docs = mongo.get({}, "CareerData")

    # Feature Engineering
    fe = featureEngineering()
    X = np.array([fe.extract_features_for_xgboost(doc) for doc in raw_docs])
    y = np.array([1 if len(doc.get("experiences", doc.get("career_history", []))) > 1 else 0 for doc in raw_docs])

    print(f"Shape von X: {X.shape}")
    print(f"Shape von y: {y.shape}")

    if len(X) == 0:
        print("Keine gültigen Daten für Training gefunden.")
        return

    # Train/Validation Split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

    print(f"Trainingsdaten: {X_train.shape}, Validierungsdaten: {X_val.shape}")

    # Modell trainieren
    model = XGBoostModel()
    model.train(X_train, y_train)

    # Evaluieren
    model.evaluate(X_val, y_val, show_report=True)

    # Modell speichern mit Timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f"saved_models/xgboost_model_{timestamp}.joblib"
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    joblib.dump(model.model, model_path)
    print(f"Modell gespeichert unter: {model_path}")

run_xgboost_pipeline()

Starte XGBoost-Pipeline...
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Keine gültigen Erfahrungen mit Position und Startdatum gefunden
Shape von X: (2247, 51)
Shape von y: (2247,)
Trainingsdaten: (1572, 51), Validierungsdaten: (675, 51)
F1 Score:     0.9992
Accuracy:     0.9985
Klassifikationsbericht:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        19
           1       1.00      1.00      1.00       656

    accuracy                           1.00       675
   macro av

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [18:30:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [22]:
def predict(input_sequence, model_path="saved_models/xgboost_model_20250415_183015.joblib", expected_steps=51):
    model = joblib.load(model_path)

    expected_features_per_step = 3
    expected_steps_count = expected_steps // expected_features_per_step  # 17 Schritte

    # Sicherstellen, dass jeder Eintrag 3 Features hat
    padded = input_sequence[:expected_steps_count]  # kürzen auf 17
    while len(padded) < expected_steps_count:
        padded.append([0, 0, 0])  # auffüllen mit 0er Steps

    # In flaches Array umwandeln
    input_array = np.array(padded, dtype=np.float32).flatten().reshape(1, -1)

    prob = model.predict_proba(input_array)[0][1]
    status = "wechselbereit" if prob > 0.5 else "bleibt wahrscheinlich"
    return float(prob), status

In [23]:
example_input = [[24, 2, 1], [36, 3, 1], [12, 3, 1]]  # z. B. [Dauer in Monaten, Level, Branche]
prob, status = predict(example_input)
print(f"Wechselwahrscheinlichkeit: {prob:.2f} → Einschätzung: {status}")

Wechselwahrscheinlichkeit: 0.90 → Einschätzung: wechselbereit
